In [11]:
import numpy as np
import os
import torch
import sys
from misc.example_helper import *
import importlib
import processing
import models
%load_ext autoreload
%autoreload 2
importlib.reload(processing)
importlib.reload(models)
from processing.data_manager import SHREDDataManager
from processing.parametric_data_manager import ParametricSHREDDataManager
from models.shred_models import SHRED
from models.shred_models import evaluate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import os

# Initialize SHREDDataManager
manager = SHREDDataManager(
    lags = 20,
    train_size = 0.7,
    val_size = 0.15,
    test_size = 0.15,
    compression=False,
    method="all"
)

file_path = "datasets\sea_surface_temperature\SST_data.npy"
manager.add(
    data=file_path,
    random_sensors=3,
    id = "SST"
)

In [24]:
manager.sensor_summary

,field id,sensor id,sensor type,location/trajectory
0,SST,SST-0,stationary (randomly selected),"(57, 137)"
1,SST,SST-1,stationary (randomly selected),"(85, 311)"
2,SST,SST-2,stationary (randomly selected),"(86, 230)"


In [25]:
manager.sensor_measurements

,time,SST-0,SST-1,SST-2
0,0,19.850000,27.109999,25.389999
1,1,19.370000,27.269999,25.439999
2,2,19.110000,27.019999,25.549999
3,3,18.550000,26.899999,26.039999
4,4,17.950000,27.019999,26.359999
...,...,...,...,...
1395,1395,27.889999,28.219999,26.089999
1396,1396,27.549999,27.839999,24.719999
1397,1397,27.279999,27.589999,24.929999
1398,1398,25.749999,28.269999,25.939999


In [26]:
# Get train/val/test datasets
train_dataset, val_dataset, test_dataset = manager.preprocess()

In [ ]:
# initialize SHRED
shred = SHRED(sequence='LSTM', decoder='SDN')
# fit SHRED
val_errors = shred.fit(train_dataset, val_dataset, num_epochs=1000)


Fitting Reconstructor...


Epoch 2/1000:  19%|█▉        | 3/16 [00:00<00:02,  5.80batch/s, loss=0.0344, L2=0.482]

KeyboardInterrupt: 

In [20]:
evaluate(model = shred, test_dataset=test_dataset, data_manager=manager)

,SST
reconstruction,0.019084
prediction,0.040854
forecast,0.042119
sensor_forecast,0.103373


In [22]:
reconstruction = shred.reconstruct(test_dataset.reconstructor_dataset.X)
reconstruction.shape

(211, 50)